In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#hide
from RS.core import *
from RS.model import *

In [ ]:
from fastai.vision.all import *

In [ ]:
df=pd.read_csv('./train.csv')

In [ ]:
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(RsImage.create)),
                       TransformBlock(type_tfms=partial(RsMask.create)),
                      ),
               get_x=ColReader('path'),
               get_y=ColReader('path'),
               splitter=ColSplitter('valid'),
               item_tfms=[aug,aug2]
              )

# db.summary(source=df)

In [ ]:

dls = db.dataloaders(source=df,bs=1, num_workers=8)


In [ ]:
model=ResNestedUNet(num_classes=10,input_channels=4,deep_supervision=False)

In [ ]:
states=torch.load('./models/resunet.pth')

In [ ]:
model.load_state_dict(states)

<All keys matched successfully>

In [ ]:
model.cuda().eval()

In [ ]:
import tqdm

In [ ]:
def IOU( pred,label, num_classes=10):
    pred = F.softmax(pred, dim=1)              
    pred = torch.argmax(pred, dim=1).squeeze(1)
    present_iou_list = np.zeros(num_classes)
    pred = pred.view(-1)
    label = label.view(-1)
    # Note: Following for loop goes from 0 to (num_classes-1)
    # and ignore_index is num_classes, thus ignore_index is
    # not considered in computation of IoU.
    for sem_class in range(num_classes):
        if sem_class!=30:
            pred_inds = (pred == sem_class)
            target_inds = (label == sem_class)
            if target_inds.long().sum().item() == 0:
                iou_now = float('nan')
            else: 
                intersection_now = (pred_inds[target_inds]).long().sum().item()
                union_now = pred_inds.long().sum().item() + target_inds.long().sum().item() - intersection_now
                iou_now = float(intersection_now) / float(union_now)
                present_iou_list[sem_class]=iou_now
    return present_iou_list

In [ ]:
ioussmall=[]
iousbig=[]
iousnormal=[]
iouss=[]
IOUn=[]
IOUs=[]
IOUb=[]
Iouf=[]

for img,label in tqdm.tqdm(dls.valid):
    with torch.no_grad():
        result=model(img)
        resultsmall=model(F.avg_pool2d(img,2))
        resultbig=model(F.upsample_bilinear(img,scale_factor=2))
    resultsmall=F.upsample_nearest(resultsmall,scale_factor=2)
    resultbig=F.upsample_nearest(resultbig,scale_factor=0.5)
    temp=IOU(resultsmall,label)
    ioussmall.append(temp)
    IOUs.append(np.nanmean(temp[temp!=0]))
    temp=IOU(resultbig,label)
    iousbig.append(temp)
    IOUb.append(np.nanmean(temp[temp!=0]))
    temp=IOU(result,label)
    iousnormal.append(temp)
    IOUn.append(np.nanmean(temp[temp!=0]))
    final=torch.cat((result,resultsmall,result)).max(dim=0)[0].unsqueeze(0)    
    
    temp=IOU(final,label)
    iouss.append(temp)
    Iouf.append(np.nanmean(temp[temp!=0]))

100%|██████████| 1000/1000 [01:04<00:00, 15.50it/s]


In [ ]:
np.mean(IOUn),np.mean(IOUs),np.mean(IOUb),np.mean(Iouf)

(0.5653191461106647,
 0.49684722509666857,
 0.49731780533390835,
 0.5548164931745874)

In [ ]:
np.asarray(ioussmall).mean(axis=0)

array([3.68022865e-01, 8.24807005e-01, 1.23681789e-03, 6.08031373e-02,
       2.11516339e-03, 7.58265864e-02, 1.06529124e-02, 1.13778025e-02,
       1.22728056e-01, 2.83816128e-04])

In [ ]:
np.asarray(iousbig).mean(axis=0)

array([0.4495213 , 0.84564855, 0.00621638, 0.10307262, 0.00786099,
       0.1602063 , 0.01822766, 0.01834135, 0.18301341, 0.00772093])

In [ ]:
np.asarray(iousnormal).mean(axis=0)

array([0.52416253, 0.87215355, 0.02077488, 0.13245133, 0.00933888,
       0.23663429, 0.02253046, 0.02746339, 0.20179011, 0.01082341])

In [ ]:
import skimage.io as skio

In [ ]:
imgs=get_files('/home/staff/xin/Downloads/tianchi/test/')

In [ ]:
for img in imgs:
    temp=RsImage.create(img)
    temp=val_pipe(image=temp.numpy())['image']
    with torch.no_grad():
        result=model(temp.unsqueeze(0).cuda())
#     result=F.avg_pool2d(result,2)
    result=result.argmax(axis=1)
    result=result[0]+1
    result=result.cpu().numpy().astype('uint8')
    newname='/home/staff/xin/Downloads/tianchi/resultV0/'+img.name.replace('tif','png')
    skio.imsave(newname,result)

In [ ]:
.shape

(1, 4, 512, 512)

In [ ]:
img.shape

(1, 4, 256, 256)